In [35]:
from pyspark.sql import SparkSession
#used to read from Met's object API
import requests
import time
from IPython.display import clear_output
import math

In [33]:
spark = SparkSession.builder.master("local[4]").appName("Extract").config("spark.ui.port", '4050').getOrCreate()
sc = spark.sparkContext

24/12/15 16:34:51 WARN Utils: Your hostname, Jacobs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.104 instead (on interface en0)
24/12/15 16:34:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/15 16:34:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
ids = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/objects').json()

491992


In [81]:
ids_rdd = sc.parallelize(ids["objectIDs"])

In [82]:
n = 5000
ids_sample = ids_rdd.sample(withReplacement=False, fraction=(n/ids["total"]), seed=1)

In [ ]:
id_list = list(ids_sample.collect())

5048


In [84]:
# quick and dirty solution: make request, then sleep for 1/80 seconds
# build a list of JSON objects
# there are 471,581 objects in total, so I'm going to evenly pick out objects from this range
# lots of empty fields, so after data cleaning I'll probably have closer to 10,000 left
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/'
objects = []
i = 0
id_total = len(id_list)
for id in id_list:
    i += 1
    clear_output(wait=True)
    path = url+str(id)
    print("Collecting object {a}/{b}\nObject ID: {c}".format(a=i, b=id_total, c=id))
    request = requests.get(path)
    obj_dict = request.json()
    if "message" in obj_dict:
        print(obj_dict)
        break
    else:
        objects.append(obj_dict)
    time.sleep(float(1/80))

Object ID: 919579


In [89]:
def getFields(obj:dict):
    # The fields I want to keep
    keys = ['objectID', 'isHighlight', 'accessionYear', 'department', 'objectName', 
            'culture','artistNationality', 'artistGender', 'objectEndDate', 'isTimelineWork']
    new_obj = {}
    for key in obj:
        if key in keys:
            new_obj[key] = obj[key]
    return new_obj

In [91]:
objects_rdd = sc.parallelize(objects)

In [92]:
objects_rdd = objects_rdd.map(lambda x: getFields(x))

In [94]:
objects_rdd.collect()

24/12/15 21:07:48 WARN TaskSetManager: Stage 8 contains a task of very large size (1749 KiB). The maximum recommended task size is 1000 KiB.


[{'objectID': 14,
  'isHighlight': False,
  'accessionYear': '1979',
  'department': 'The American Wing',
  'objectName': 'Coin',
  'culture': '',
  'artistNationality': 'American',
  'artistGender': '',
  'objectEndDate': 1907,
  'isTimelineWork': False},
 {'objectID': 108,
  'isHighlight': False,
  'accessionYear': '1989',
  'department': 'The American Wing',
  'objectName': 'Andiron',
  'culture': 'American',
  'artistNationality': 'American',
  'artistGender': '',
  'objectEndDate': 1814,
  'isTimelineWork': False},
 {'objectID': 366,
  'isHighlight': False,
  'accessionYear': '1946',
  'department': 'The American Wing',
  'objectName': 'Basket',
  'culture': '',
  'artistNationality': '',
  'artistGender': '',
  'objectEndDate': 1890,
  'isTimelineWork': False},
 {'objectID': 540,
  'isHighlight': False,
  'accessionYear': '1946',
  'department': 'The American Wing',
  'objectName': 'Bitters bottle',
  'culture': 'American',
  'artistNationality': '',
  'artistGender': '',
  'obje

In [95]:
df = objects_rdd.toDF()

24/12/15 21:16:18 WARN TaskSetManager: Stage 9 contains a task of very large size (1749 KiB). The maximum recommended task size is 1000 KiB.


In [107]:
import os
path = os.getcwd() + "/extractedDataset"
df.repartition(1).write.csv(path, mode='overwrite', header=True)

24/12/15 21:53:09 WARN TaskSetManager: Stage 19 contains a task of very large size (1749 KiB). The maximum recommended task size is 1000 KiB.


In [108]:
spark.stop()